In [3]:
''' RNN Bidireccional en Keras '''
__author__ = '@Tssp'
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from codvidutils.Transformation_class import Transformation
import tensorflow as tf
from keras.layers import Conv2D, Input, Dense, MaxPooling2D, UpSampling2D, Flatten, Dropout
from tensorflow.keras.regularizers import l2
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

In [2]:
trns = Transformation('data/train_split_v4.csv', 'data/test_split_v4.csv')
#------------Read images and prepare dataset------------#
train_class, test_class = trns.read_imgs_paths()
X_train, X_test, diseaseID_train, diseaseID_test = trns.read_imgs(train_class, test_class)
del train_class, test_class
#------------Imbalanced methods------------#
X_train, X_test, diseaseID_train = trns.underbalance(X_train, X_test, diseaseID_train)
print('X_train.shape: {}\nX_test.shape: {}'.format(X_train.shape, X_test.shape))
print("Normal train: ",diseaseID_train[diseaseID_train==0].shape)
print("Pneumonia train: ",diseaseID_train[diseaseID_train==2].shape)
print("COVID train: ",diseaseID_train[diseaseID_train==1].shape)
print("*******************************************************")
print("Normal test: ",diseaseID_test[diseaseID_test==0].shape)
print("Pneumonia test: ",diseaseID_test[diseaseID_test==2].shape)
print("COVID test: ",diseaseID_test[diseaseID_test==1].shape)
X_train, X_test, diseaseID_train, diseaseID_test = trns.new_imgs(X_train, X_test, diseaseID_train, diseaseID_test)
Y_train = np.copy(diseaseID_train)
del diseaseID_train
Y_train[Y_train==2]=0
Y_test = np.copy(diseaseID_test)
Y_test[Y_test==2]=0

You are going to read from: data/train_split_v4.csv and data/test_split_v4.csv
Total number of images: 1540
shape X: 13600 1540,  disease_ID (Y): 13600 1540
Count of classes:  Counter({0: 7966, 2: 5447, 1: 187})
New diseaseID shape:  (9097,)
New X shape:  (9097, 120000)
New Count of classes:  Counter({0: 4500, 2: 4500, 1: 97})
Undersample shapes:
diseaseID_train.shape: (9097,)
X_train.shape: (9097, 120000)
X_train.shape: (9097, 200, 200, 3)
X_test.shape: (1540, 200, 200, 3)
Normal train:  (4500,)
Pneumonia train:  (4500,)
COVID train:  (97,)
*******************************************************
Normal test:  (885,)
Pneumonia test:  (591,)
COVID test:  (64,)
(2425, 180, 180, 3)
(2425,)
X_train.shape:  (11522, 180, 180, 3)
diseaseID_train.shape:  (11522,)
X_test_news.shape:  (1600, 180, 180, 3)
diseaseID_test_news.shape:  (1600,)
X_test.shape:  (3140, 180, 180, 3)
diseaseID_test.shape:  (3140,)


In [ ]:
def autoencoder(p_drop, p_l2):
    input_img = Input(shape=X_train.shape[1:])
    encoder = Dropout(rate = p_drop)(input_img)
    encoder = Conv2D(256, (3, 3), activation='relu', padding='same', name='Econv2d_1')(input_img)
    encoder = MaxPooling2D((2,2), padding='same', name='Emaxpool2d_1')(encoder)
    encoder = Dropout(rate = p_drop)(encoder)
    encoder = Conv2D(128, (3, 3), activation='relu', padding='same', name='Econv2d_2')(encoder)
    encoder = MaxPooling2D((2,2), padding='same', name='Emaxpool2d_2')(encoder)
    encoder = Dropout(rate = p_drop)(encoder)
    encoder = Conv2D(64, (3, 3), activation='relu', padding='same', name='Econv2d_3')(encoder)
    encoder = MaxPooling2D((2,2), padding='same', name='Emaxpool2d_3')(encoder)

    encoder = Dropout(rate = p_drop)(encoder)
    decoder = Conv2D(64, (3,3), activation='relu', padding='same', name='Dconv2d_1')(encoder)
    decoder = UpSampling2D((2, 2), name='Dupsamp_1')(decoder)
    encoder = Dropout(rate = p_drop)(encoder)
    decoder = Conv2D(128, (3, 3), activation='relu', padding='same', name='Dconv2d_2')(decoder)
    decoder = UpSampling2D((2, 2), name='Dupsamp_2')(decoder)
    decoder = Conv2D(256, (3, 3), activation='relu', name='Dconv2d_3')(decoder)
    decoder = UpSampling2D((2, 2), name='Dupsamp_3')(decoder)
    decoder = Conv2D(3, (3, 3), activation='sigmoid', padding='same', name='Dconv2d_out')(decoder)

    autoencoder = Model(input_img, decoder)
    autoencoder.summary()